In [1]:
import os
import re
import urllib
import time
import pickle
import numpy as np
import pandas as pd

import asyncio
import aiohttp

from urllib.request import urlopen
from bs4 import BeautifulSoup

In [4]:
import nest_asyncio
nest_asyncio.apply()

В данном ноутбуке реализованы функции парсинга информации о блюдах со специального сайта про кулинарию. В результате работы формируется Dataset. 

In [5]:
# Константные значения
CLASSES = ["Завтрак", "Бульон", "Закуска", "Напиток", "Основное блюдо",
           "Паста или пицца", "Ризотто", "Салат", "Соус",
           "Суп", "Сэндвич", "Выпечка", "Заготовка"]
URL_CLASSES = ["zavtraki", "bulony", "zakuski", "napitki", "osnovnye-blyuda",
               "pasta-picca", "rizotto", "salaty", "sousy-marinady",
               "supy", "sendvichi", "vypechka-deserty", "zagotovki"]

URL = "https://eda.ru/recepty/"
MAIN_URL = "https://eda.ru"

## Parsing links about products

Парсинг для нахождения ссылок на все блюда

In [6]:
async def get_html(url):
    """
    Получение HTML страницы
    """
    HEADERS = {'User-Agent': 'Mozilla/5.0'}
    async with aiohttp.ClientSession() as session:
        async with session.get(url, headers=HEADERS) as resp:
            return await resp.text()


async def find_all_product_class(class_name, links_dict):    
    """
    Находим ссылки блюд конкретного класса
    """
    url_section = os.path.join(URL, class_name)
    products = []
    page = 1
    
    while True: 
        # Проверяем открытие HTML документа
        loop = asyncio.get_event_loop()
        try:
            html = loop.run_until_complete(get_html(url_section + f'?page={page}'))
        except:
            raise f"Проблема со ссылкой Url. Скрипт остановлен!"
        
        # Парсим ссылки на блюда со страницы
        bs = BeautifulSoup(html, 'html.parser')
        grid_products = bs.findAll('div', {'class': {'emotion-1f6ych6'}})
        if len(grid_products) == 0:
            break
        
        # Сохраняем в список ссылки данной страницы
        for grid in grid_products:
            products.append(os.path.join(URL, grid.find('a', {'class': {'emotion-12sjte8'}})['href']))
        
        page += 1
        
    print(f"{class_name} is done: pages {page - 1} products {len(products)}")
    links_dict[class_name] = products


async def get_all_product():
    """
    Находим ссылки блюд каждого класса
    """
    start_time = time.time()
    links_dict = {}
    
    # Выполняем поиск ссылок по страницам категорий асинхронно
    loop = asyncio.get_event_loop()
    tasks = []
    for i in range(len(URL_CLASSES)):
        tasks.append(loop.create_task(find_all_product_class(URL_CLASSES[i], links_dict)))
    
    # Ожидаем результат функций и записываем результат
    links, labels = [], []
    for i, task in enumerate(tasks):
        await task
        
        links.extend(links_dict[URL_CLASSES[i]])
        labels.extend([CLASSES[i] for _ in range(len(links_dict[URL_CLASSES[i]]))])
        
    time_difference = time.time() - start_time
    print(f'Scraping time: %.2f seconds.' % time_difference)
    return links, labels

In [9]:
loop = asyncio.get_event_loop()
links, labels = loop.run_until_complete(get_all_product())

zagotovki is done: pages 8 products 103
vypechka-deserty is done: pages 714 products 9996
sendvichi is done: pages 46 products 633
supy is done: pages 255 products 3559
sousy-marinady is done: pages 104 products 1443
salaty is done: pages 391 products 5469
rizotto is done: pages 24 products 326
pasta-picca is done: pages 140 products 1958
osnovnye-blyuda is done: pages 714 products 9996
napitki is done: pages 166 products 2318
zakuski is done: pages 441 products 6161
bulony is done: pages 7 products 95
zavtraki is done: pages 164 products 2289
Scraping time: 4463.76 seconds.


In [13]:
data = {"links": links, "classes": labels}

with open('links.pickle', 'wb') as f:
    pickle.dump(data, f)

## Parsing information about product

In [7]:
with open('links.pickle', 'rb') as f:
    data = pickle.load(f)
links = data['links']
labels = data['classes']

Прасинг ссылок блюд с информацией о способе приготовления, фото и так далее

In [8]:
def parsing_product_link(links, labels, data_product):
    """
    Парсинг блюд
    """
    start_time = time.time()
    ENG_NAME_CONST = ("Калорий", "Белки", "Жиры", "Углеводы")
    ENG_FORMAT_CONST = ("ккал", "грамм", "грамм", "грамм")
    
    for i in range(len(links)):        
        cur_link = MAIN_URL + links[i]
        
        # Проверяем открытие HTML документа
        try:
            html = urlopen(cur_link)
        except:
            print(f"Проблема со ссылкой {cur_link}")
            continue
        
        bs = BeautifulSoup(html, 'html.parser')
        
        # Парсинг названия блюда
        parse = bs.find('h1', {'class': {'emotion-gl52ge'}})
        data_product['name'].append(parse.text.replace("\xa0", " "))
        
        # Парсинг инструкция приготовления
        parse = bs.findAll('span', {'class': {'emotion-1dvddtv'}})
        cur_text = " ".join([elem.find('span').text for elem in parse])
        data_product['text'].append(cur_text.replace("\xa0", " "))
        
        # Парсинг состав ингредиентов
        parse1 = bs.findAll('span', {'itemprop': {'recipeIngredient'}})
        parse2 = bs.findAll('span', {'class': {'emotion-bsdd3p'}})
        cur_ing = " ".join([f"{parse1[i].text}: {parse2[i].text}," for i in range(len(parse1))])
        data_product['ingredient'].append(cur_ing.rstrip(','))
        
        # Парсинг состав БЖУ
        parse = bs.findAll('div', {'class': {'emotion-8fp9e2'}})
        cur_eng = " ".join([f"{ENG_NAME_CONST[i]} {parse[i].text} {ENG_FORMAT_CONST[i]}," for i in range(4)])
        data_product['energy'].append(cur_eng.rstrip(','))
        
        # Парсинг время готовки
        parse = bs.find('div', {'class': {'emotion-my9yfq'}})
        data_product['time_cook'].append(parse.text)
        
        # Парсинг тип кухни
        parse = bs.findAll('span', {'class': {'emotion-1h6i17m'}})
        data_product['type_kitchen'].append(parse[2].text)
        
        # Ссылка
        data_product['link'].append(cur_link)
        
        # Парсинг метка класса
        data_product['label'].append(labels[i])
        
        if i % 1000 == 0:
            print(f"Parsing {i} / {len(links)}")
            
    time_difference = time.time() - start_time
    print(f'Scraping time: %.2f seconds.' % time_difference)

In [44]:
data_product = {"name": [], "text": [], "ingredient": [], "energy": [],
                "time_cook": [], "type_kitchen": [], "link": [], "label": []}

In [1]:
parsing_product_link(links, labels, data_product)

## Create Dataset CSV File

In [2]:
df = pd.DataFrame(data_product)

In [50]:
df.to_csv('food-dataset-ru.csv', index=False)

## Preprocessing Dataset

In [36]:
df = pd.read_csv('food-dataset-ru.csv')

Обрабатываем пропуски

In [38]:
df[df['text'].isna()]

,name,text,ingredient,energy,time_cook,type_kitchen,label
5999,Жюльен с пореем и грибами в соусе бешамель,NaN,"Шампиньоны: 300 г, Чеснок: 2 зубчика, Лук-поре...","Калорий 338 ккал, Белки 10 грамм, Жиры 29 грам...",Европейская кухня,45 минут,Закуска
23801,Фасолевый салат,NaN,"Фасоль: 675 г, Соль: по вкусу, Петрушка: 5 г, ...","Калорий 26 ккал, Белки 2 грамм, Жиры 0 грамм, ...",Болгарская кухня,3 часа,Салат


In [39]:
# Удаляем рецепты к которым отсутствуют интсрукции
df = df[~df['text'].isna()]

Удаляем дубликаты

In [41]:
df.groupby(['name', 'text']).agg({"label": 'count'}).reset_index().sort_values('label', ascending=False)

,name,text,label
33238,Украинский охотничий суп из куриных потрошков,Куриные потроха залить холодной водой и довест...,5
24845,Салат из маринованного огурца и кунжута,Обжарить семечки кунжута на предварительно раз...,5
37637,​Десерт «Монблан»,"Смешайте муку с сахаром и солью, добавьте слив...",4
8344,Индейка с каштанами,На каждом каштане с его тупой стороны сделать ...,4
27735,Слоеный куриный салат с опятами,"Курицу отварить, охладить и нарезать соломкой....",4
...,...,...,...
13536,Курица с овощами и куркумой,Разрезать красный лук на четвертинки. Если исп...,1
13535,Курица с овощами гриль и песто,Разогрейте сковороду гриль. Обмакните колечки ...,1
13534,Курица с овощами в соевом соусе,Куриное филе нарезать средними кусочками. Крас...,1
13533,Курица с овощами в соево-медовом соусе,Чеснок очистить и мелко нарезать и отправить в...,1


In [42]:
df = df.drop_duplicates()

In [43]:
# Проверим
df.groupby(['name', 'text']).agg({"label": 'count'}).reset_index().sort_values('label', ascending=False)

,name,text,label
0,100% цельнозерновой хлеб,Смажьте маслом две хлебные формы. Сделайте опа...,1
25104,Салат из редьки с овощами,"Овощи нарезать соломкой, укроп измельчить. Все...",1
25088,Салат из раков с черносливом,Соедините раков с вареными овощами. Добавьте б...,1
25089,Салат из раковых шеек,"Сварить морковь, картофель и цветную капусту о...",1
25090,Салат из раковых шеек в авокадо,"Авокадо разрезать на две половинки, удалить ко...",1
...,...,...,...
12546,Куриная грудка в фольге под соусом,"Смешиваем все, кроме сыра и куриного филе. Пол...",1
12547,"Куриная грудка запеченная с розмарином, тимьян...",Приготовить смесь из размягченного сливочного ...,1
12548,Куриная грудка кордон блю в беконе,"Смешиваем тертые сыры — моцареллу, грюйер и па...",1
12549,Куриная грудка на овощной подушке,"Грудки промыть, сделать небольшие надрезы. Доб...",1


In [46]:
df.to_csv('food-dataset-ru.csv', index=False, encoding='utf-8-sig')